In [84]:

import os
import json
import csv
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [85]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [86]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [87]:

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")


In [ ]:
def record_booking(name, trip_date, amount):
    booking_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Get the project root directory (where the notebook is located)
    project_root = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
    booking_file_path = os.path.join(project_root, "booking.txt")
    
    # Check if file exists to determine if we need to write headers
    file_exists = os.path.exists(booking_file_path)
    
    # Open file in append mode
    with open(booking_file_path, "a", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        
        # Write headers if file is new
        if not file_exists:
            writer.writerow(["booking_date", "name", "travel_date", "amount_paid"])
        
        # Write booking data
        writer.writerow([booking_date, name, trip_date, amount])
    
    return f"Booking processed successfully for {name} on {trip_date} for {amount} to {booking_file_path}"


In [ ]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

booking_function = {
    "name": "record_booking",
    "description": "Record booking for a trip. Call this whenever you need to make booking for a user, for example when a customer says 'I need to make a booking' or when you ask them if they need to make a booking and they confirmed yes",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The name of the customer",
            },
             "trip_date": {
                "type": "string",
                "format": "date-time",
                "description": "The date on which the customer wants to make the trip.",
            },
             "amount": {
                "type": "number",
                "description": "The cost of the ticket for the trip",
            },
        },
        "required": ["name", "trip_date", "amount"],
        "additionalProperties": False
    },
"strict": True
}

tools = [{"type": "function", "function": price_function}, {"type": "function", "function": booking_function}]

In [68]:
def call_function(name, args):
    if name == "get_ticket_price":
        additional_message = args["destination_city"]
        result = get_ticket_price(**args)
    elif name == "record_booking":
        result = record_booking(**args)
        additional_message = ""
    else:
        raise ValueError(f"unknown function {name}")
    return result, additional_message

In [ ]:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    print(tool_call)
    arguments = json.loads(tool_call.function.arguments)
    result, additional_message = call_function(tool_call.function.name, arguments)
    response = {
        "role": "tool",
        "content": str(result),
        "tool_call_id": tool_call.id
    }
    return response, additional_message

ChatCompletionMessageFunctionToolCall(id='call_vprMsOQNeongeFosaj7wdVIJ', function=Function(arguments='{"destination_city":"London"}', name='get_ticket_price'), type='function')
Tool get_ticket_price called for London
ChatCompletionMessageFunctionToolCall(id='call_u7ulz0NkQN2GAM3ohjyz67gD', function=Function(arguments='{"name":"tobi adeoye","trip_date":"2050-10-25T00:00:00Z","amount":799}', name='record_booking'), type='function')


In [ ]:
import base64
from io import BytesIO
from PIL import Image

def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
image = artist("New York City")
display(image)

In [ ]:
from pydub import AudioSegment
from pydub.playback import play

def talker(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="alloy",    # Also, try replacing onyx with alloy
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [65]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        if city:
            image = artist(city)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Comment out or delete the next line if you'd rather skip Audio for now..
    talker(reply)
    
    return history, image

In [69]:
# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7897
* To create a public link, set `share=True` in `launch()`.


ChatCompletionMessageFunctionToolCall(id='call_YUL8oVkBS6nT3TwnUcZUdRGD', function=Function(arguments='{"destination_city":"London"}', name='get_ticket_price'), type='function')
{'destination_city': 'London'}
Tool get_ticket_price called for London


Traceback (most recent call last):
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/route_utils.py", line 349, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 2274, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 1781, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/anyio/to_thread.py", line 56, in run_sync
    return